In [28]:
import pandas as pd
import os


def load_and_combine_pkl_files(directory_path):
    # List to hold all the dataframes
    dataframes = []

    # Loop through all the files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith("1103_1203_logits_qa_postprocess_inferenced_df.pkl"):
            # Construct full file path
            file_path = os.path.join(directory_path, filename)
            # Load the dataframe from a pkl file
            df = pd.read_pickle(file_path)
            # Append the dataframe to the list
            dataframes.append(df)

    # Concatenate all dataframes into one big dataframe
    big_df = pd.concat(dataframes, ignore_index=True)

    return big_df


# Usage
directory_path = '/Users/ximing/Desktop/Explainprompt'
big_df = load_and_combine_pkl_files(directory_path)
big_df = big_df
print(big_df)


                                               prompt  \
0   Who wrote the Georgics? Respond in the form of...   
1   Where did Augustus die? Respond in the form of...   
2   Where do the ibex live? Respond in the form of...   
3   Who died in the mutiny? Respond in the form of...   
4   When was Vieira killed? Respond in the form of...   
..                                                ...   
93  Who was emir of Qazvin? Respond in the form of...   
94  What are commonly used? Respond in the form of...   
95  What relocated in 1934? Respond in the form of...   
96  What was built in 1960? Respond in the form of...   
97  What does AF stand for? Respond in the form of...   

                                          real_output  \
0   The Georgics, a didactic poem in four books, i...   
1   In the waning days of summer, the great Roman ...   
2   In the rugged and unforgiving landscapes of th...   
3   In the year 1857, amidst British rule in India...   
4   Vieira was killed on Augus

In [29]:
print(big_df.columns)

Index(['prompt', 'real_output', 'token_level', 'word_level', 'component_level',
       'instruction', 'query', 'component_range', 'instruction_weight',
       'query_weight', 'exec_time', 'input_cost', 'output_cost', 'total_cost',
       'real_ouputs', 'instructions_tokens', 'query_tokens'],
      dtype='object')


In [30]:
expanded_df = pd.DataFrame([word for sublist in big_df['instructions_tokens'] for word in sublist])


In [31]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  # 计算总和和均值
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value']  # 重新命名列以便阅读
print(grouped_stats)

     token  total_value  average_value
0  Respond     7.035245       0.071788
1        a     6.954704       0.070966
2     form     6.981214       0.071237
3       in     6.979714       0.071222
4     long     6.955979       0.070979
5       of     6.989855       0.071325
6    story     6.931501       0.070730
7      the     6.994080       0.071368


discretize 

In [27]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value'] 
print(grouped_stats)

     token  total_value  average_value
0  Respond     6.650957       0.067867
1        a     6.684485       0.068209
2     form     6.512113       0.066450
3       in     6.794886       0.069336
4     long     7.163081       0.073093
5       of     6.620741       0.067559
6    story     7.846718       0.080069
7      the     6.574434       0.067086


simliarity 

In [21]:
grouped_stats = expanded_df.groupby('token').agg({
    'value': ['sum', 'mean']  # 计算总和和均值
}).reset_index()
grouped_stats.columns = ['token', 'total_value', 'average_value']  # 重新命名列以便阅读
print(grouped_stats)

     token  total_value  average_value
0  Respond     4.941783       0.050426
1        a     5.138980       0.052439
2     form     5.004630       0.051068
3       in     5.109278       0.052135
4     long     5.712436       0.058290
5       of     5.075337       0.051789
6    story     7.830939       0.079908
7      the     4.838620       0.049374


In [ ]:
print(big_df.iloc[4]['word_level'])

In [ ]:
print(big_df['bottom_reconstructed_query_0.2_result'][:20])


In [11]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.07142857142857142


In [12]:
differences = big_df['real_output'] != big_df['top_reconstructed_query_0.2_result']
print(differences.sum()/len(differences))

0.35714285714285715


In [11]:
differences = big_df['real_output'] != big_df['bottom_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

0.030612244897959183


In [14]:
differences = big_df['real_output'] != big_df['top_reconstructed_instruction_0.2_result']
print(differences.sum()/len(differences))

1.0


In [15]:
print(big_df['instruction_weight'].mean())

0.8787098084681545


In [16]:
print(big_df['query_weight'].mean())

0.12129019153184557


In [12]:
print(big_df['exec_time'].mean())

17.86105881661785


In [13]:
print(big_df['input_cost'].mean())

1426.1836734693877


In [14]:
print(big_df['output_cost'].mean())


154.48979591836735


In [15]:
print(big_df['total_cost'].mean())


1580.6734693877552
